In [ ]:
import pandas as pd

df = pd.read_csv('preprocessed_df.csv', delimiter=' ',names=['id', 'col1', 'col2', 'col3', 'col4', 'col5'])

total_rows = len(df)
rows_to_keep = int(total_rows * 0.8)

df_80 = df.iloc[:rows_to_keep]

df_80.to_csv('training_data.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('training_data.csv', delimiter=' ')

row_count = len(df)
print(f"Number of rows: {row_count}")

Number of rows: 1775842


In [ ]:
import pandas as pd

df = pd.read_csv('preprocessed_df.csv', delimiter=' ')

row_count = len(df)
print(f"Number of rows: {row_count}")

Number of rows: 2219802


In [ ]:
import pandas as pd

df = pd.read_csv('preprocessed_df.csv', delimiter=' ', 
                 names=['id', 'col1', 'col2', 'col3', 'col4', 'col5'])

total_rows = len(df)
start_index = int(total_rows * 0.8)

df_test = df.iloc[start_index:]

df_test.to_csv('testing_data.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('testing_data.csv', delimiter=' ')

row_count = len(df)
print(f"Number of rows: {row_count}")

Number of rows: 443961


In [ ]:
import mysql.connector

try:
    db = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="anomoly"
    )
    
    cursor = db.cursor()
    
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()
    print("Tables in database:")
    for table in tables:
        print(table[0])
        
    cursor.execute("DESCRIBE sensor_readings;")
    columns = cursor.fetchall()
    print("\nStructure of sensor_readings table:")
    for column in columns:
        print(f"Column: {column[0]}, Type: {column[1]}")
        
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if 'db' in locals() and db.is_connected():
        cursor.close()
        db.close()
        print("\nMySQL connection closed.")

Tables in database:
sensor_readings

Structure of sensor_readings table:
Column: reading_id, Type: bigint(20)
Column: sensor_id, Type: int(11)
Column: temperature, Type: float
Column: humidity, Type: float
Column: light, Type: float
Column: timestamp, Type: timestamp

MySQL connection closed.


In [ ]:
import mysql.connector
import pandas as pd

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="anomoly"
)

cursor = db.cursor()

cursor.execute("DROP TABLE IF EXISTS sensor_readings")

create_table_sql = """
CREATE TABLE sensor_readings (
    epoch INT,
    moteid INT,
    temperature FLOAT,
    humidity FLOAT,
    light FLOAT,
    voltage FLOAT
)
"""
cursor.execute(create_table_sql)

df = pd.read_csv('training_data.csv',
                 names=['epoch', 'moteid', 'temperature', 'humidity', 'light', 'voltage'],delimiter=',', skiprows=1)

count = 0
for _, row in df.iterrows():
    sql = """INSERT INTO sensor_readings 
             (epoch, moteid, temperature, humidity, light, voltage) 
             VALUES (%s, %s, %s, %s, %s, %s)"""
    values = (int(row['epoch']), 
             int(float(row['moteid'])), 
             float(row['temperature']), 
             float(row['humidity']), 
             float(row['light']),
             float(row['voltage']))
    
    try:
        cursor.execute(sql, values)
        count += 1
        if count % 1000 == 0:  
            print(f"Inserted {count} rows")
            db.commit()  
    except mysql.connector.Error as err:
        print(f"Error at row {count}: {err}")
        continue

db.commit()
cursor.close()
db.close()

print(f"Total {count} rows inserted successfully!")

Inserted 1000 rows
Inserted 2000 rows
Inserted 3000 rows
Inserted 4000 rows
Inserted 5000 rows
Inserted 6000 rows
Inserted 7000 rows
Inserted 8000 rows
Inserted 9000 rows
Inserted 10000 rows
Inserted 11000 rows
Inserted 12000 rows
Inserted 13000 rows
Inserted 14000 rows
Inserted 15000 rows
Inserted 16000 rows
Inserted 17000 rows
Inserted 18000 rows
Inserted 19000 rows
Inserted 20000 rows
Inserted 21000 rows
Inserted 22000 rows
Inserted 23000 rows
Inserted 24000 rows
Inserted 25000 rows
Inserted 26000 rows
Inserted 27000 rows
Inserted 28000 rows
Inserted 29000 rows
Inserted 30000 rows
Inserted 31000 rows
Inserted 32000 rows
Inserted 33000 rows
Inserted 34000 rows
Inserted 35000 rows
Inserted 36000 rows
Inserted 37000 rows
Inserted 38000 rows
Inserted 39000 rows
Inserted 40000 rows
Inserted 41000 rows
Inserted 42000 rows
Inserted 43000 rows
Inserted 44000 rows
Inserted 45000 rows
Inserted 46000 rows
Inserted 47000 rows
Inserted 48000 rows
Inserted 49000 rows
Inserted 50000 rows
Inserted 

In [ ]:
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="anomoly"
)

cursor = db.cursor()


create_view_sql = """
CREATE OR REPLACE VIEW sensor_statistics AS
SELECT 
    moteid,
    AVG(temperature) as avg_temp,
    STDDEV(temperature) as stddev_temp,
    AVG(humidity) as avg_humidity,
    STDDEV(humidity) as stddev_humidity,
    AVG(light) as avg_light,
    STDDEV(light) as stddev_light,
    AVG(voltage) as avg_voltage,
    STDDEV(voltage) as stddev_voltage
FROM sensor_readings
GROUP BY moteid;
"""

try:
    cursor.execute(create_view_sql)
    print("View created successfully!")


    cursor.execute("SELECT * FROM sensor_statistics")
    results = cursor.fetchall()

    print("\nSensor Statistics:")
    print("MoteID | Avg Temp | StdDev Temp | Avg Humid | StdDev Humid | Avg Light | StdDev Light | Avg Volt | StdDev Volt")
    print("-" * 100)
    for row in results:
        print(f"{row[0]:6d} | {row[1]:8.2f} | {row[2]:10.2f} | {row[3]:8.2f} | {row[4]:11.2f} | {row[5]:8.2f} | {row[6]:11.2f} | {row[7]:8.2f} | {row[8]:10.2f}")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    cursor.close()
    db.close()

View created successfully!

Sensor Statistics:
MoteID | Avg Temp | StdDev Temp | Avg Humid | StdDev Humid | Avg Light | StdDev Light | Avg Volt | StdDev Volt
----------------------------------------------------------------------------------------------------
     1 |    35.88 |      33.65 |    34.32 |       13.88 |   156.58 |      174.71 |     2.52 |       0.17
     2 |    40.20 |      37.87 |    34.30 |       18.03 |   212.16 |      173.43 |     2.46 |       0.17
     3 |    33.72 |      31.89 |    34.79 |       13.85 |   146.05 |      149.11 |     2.52 |       0.16
     4 |    45.46 |      41.78 |    29.99 |       17.94 |   155.49 |      126.39 |     2.47 |       0.17
     6 |    27.56 |      31.59 |    28.51 |       18.78 |   453.11 |      519.48 |     2.43 |       0.20
     7 |    41.94 |      39.67 |    32.13 |       16.68 |   134.15 |      118.57 |     2.50 |       0.18
     8 |    21.41 |       3.75 |    37.48 |       71.92 |   683.79 |      735.37 |     2.65 |       0.05
     9

In [ ]:
import mysql.connector


db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="anomoly"
)

cursor = db.cursor()

procedure_sql = """
CREATE PROCEDURE detect_point_anomalies(
    IN p_moteid INT,
    IN p_temperature FLOAT,
    IN p_humidity FLOAT,
    IN p_light FLOAT,
    IN p_voltage FLOAT,
    IN threshold_stddev FLOAT
)
BEGIN
    SET @v_avg_temp = 0;
    SET @v_stddev_temp = 0;
    SET @v_avg_humidity = 0;
    SET @v_stddev_humidity = 0;
    SET @v_avg_light = 0;
    SET @v_stddev_light = 0;
    SET @v_avg_voltage = 0;
    SET @v_stddev_voltage = 0;

    IF threshold_stddev IS NULL THEN
        SET threshold_stddev = 3.0;
    END IF;

    SELECT 
        avg_temp, stddev_temp,
        avg_humidity, stddev_humidity,
        avg_light, stddev_light,
        avg_voltage, stddev_voltage
    INTO 
        @v_avg_temp, @v_stddev_temp,
        @v_avg_humidity, @v_stddev_humidity,
        @v_avg_light, @v_stddev_light,
        @v_avg_voltage, @v_stddev_voltage
    FROM sensor_statistics
    WHERE moteid = p_moteid;

    DROP TEMPORARY TABLE IF EXISTS temp_anomalies;
    CREATE TEMPORARY TABLE temp_anomalies (
        moteid INT,
        measurement_type VARCHAR(20),
        actual_value FLOAT,
        expected_value FLOAT,
        deviation FLOAT
    );

    IF ABS((p_temperature - @v_avg_temp) / NULLIF(@v_stddev_temp, 0)) > threshold_stddev THEN
        INSERT INTO temp_anomalies 
        VALUES (
            p_moteid,
            'temperature',
            p_temperature,
            @v_avg_temp,
            ABS((p_temperature - @v_avg_temp) / NULLIF(@v_stddev_temp, 0))
        );
    END IF;

    IF ABS((p_humidity - @v_avg_humidity) / NULLIF(@v_stddev_humidity, 0)) > threshold_stddev THEN
        INSERT INTO temp_anomalies 
        VALUES (
            p_moteid,
            'humidity',
            p_humidity,
            @v_avg_humidity,
            ABS((p_humidity - @v_avg_humidity) / NULLIF(@v_stddev_humidity, 0))
        );
    END IF;

    IF ABS((p_light - @v_avg_light) / NULLIF(@v_stddev_light, 0)) > threshold_stddev THEN
        INSERT INTO temp_anomalies 
        VALUES (
            p_moteid,
            'light',
            p_light,
            @v_avg_light,
            ABS((p_light - @v_avg_light) / NULLIF(@v_stddev_light, 0))
        );
    END IF;

    IF ABS((p_voltage - @v_avg_voltage) / NULLIF(@v_stddev_voltage, 0)) > threshold_stddev THEN
        INSERT INTO temp_anomalies 
        VALUES (
            p_moteid,
            'voltage',
            p_voltage,
            @v_avg_voltage,
            ABS((p_voltage - @v_avg_voltage) / NULLIF(@v_stddev_voltage, 0))
        );
    END IF;

    SELECT * FROM temp_anomalies ORDER BY deviation DESC;
    
    DROP TEMPORARY TABLE IF EXISTS temp_anomalies;
END
"""

try:

    cursor.execute("DROP PROCEDURE IF EXISTS detect_point_anomalies")
    

    cursor.execute(procedure_sql)
    print("Stored procedure created successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    cursor.close()
    db.close()

Stored procedure created successfully!


In [ ]:
import mysql.connector


db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="anomoly"
)

cursor = db.cursor()

try:

    create_anomalies_table = """
    CREATE TABLE IF NOT EXISTS anomalies (
        id BIGINT AUTO_INCREMENT PRIMARY KEY,
        moteid INT,
        measurement_type VARCHAR(20),
        actual_value FLOAT,
        expected_value FLOAT,
        deviation FLOAT,
        detection_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    cursor.execute(create_anomalies_table)
    print("Anomalies table created successfully!")

    create_trigger = """
    CREATE TRIGGER check_anomaly_before_insert 
    BEFORE INSERT ON sensor_readings
    FOR EACH ROW
    BEGIN
        DECLARE has_anomaly BOOLEAN DEFAULT FALSE;
        DECLARE temp_deviation FLOAT;
        DECLARE humid_deviation FLOAT;
        DECLARE light_deviation FLOAT;
        DECLARE volt_deviation FLOAT;
        
        -- Get statistics for comparison
        SELECT 
            ABS((NEW.temperature - avg_temp) / NULLIF(stddev_temp, 0)),
            ABS((NEW.humidity - avg_humidity) / NULLIF(stddev_humidity, 0)),
            ABS((NEW.light - avg_light) / NULLIF(stddev_light, 0)),
            ABS((NEW.voltage - avg_voltage) / NULLIF(stddev_voltage, 0))
        INTO 
            temp_deviation, humid_deviation, light_deviation, volt_deviation
        FROM sensor_statistics 
        WHERE moteid = NEW.moteid;
        
        -- Check temperature anomaly
        IF temp_deviation > 3.0 THEN
            INSERT INTO anomalies (moteid, measurement_type, actual_value, expected_value, deviation)
            SELECT 
                NEW.moteid, 
                'temperature', 
                NEW.temperature, 
                avg_temp, 
                temp_deviation
            FROM sensor_statistics 
            WHERE moteid = NEW.moteid;
            SET has_anomaly = TRUE;
        END IF;
        
        -- Check humidity anomaly
        IF humid_deviation > 3.0 THEN
            INSERT INTO anomalies (moteid, measurement_type, actual_value, expected_value, deviation)
            SELECT 
                NEW.moteid, 
                'humidity', 
                NEW.humidity, 
                avg_humidity, 
                humid_deviation
            FROM sensor_statistics 
            WHERE moteid = NEW.moteid;
            SET has_anomaly = TRUE;
        END IF;
        
        -- Check light anomaly
        IF light_deviation > 3.0 THEN
            INSERT INTO anomalies (moteid, measurement_type, actual_value, expected_value, deviation)
            SELECT 
                NEW.moteid, 
                'light', 
                NEW.light, 
                avg_light, 
                light_deviation
            FROM sensor_statistics 
            WHERE moteid = NEW.moteid;
            SET has_anomaly = TRUE;
        END IF;
        
        -- Check voltage anomaly
        IF volt_deviation > 3.0 THEN
            INSERT INTO anomalies (moteid, measurement_type, actual_value, expected_value, deviation)
            SELECT 
                NEW.moteid, 
                'voltage', 
                NEW.voltage, 
                avg_voltage, 
                volt_deviation
            FROM sensor_statistics 
            WHERE moteid = NEW.moteid;
            SET has_anomaly = TRUE;
        END IF;
        
        -- If it's an anomaly, prevent insertion into main table
        IF has_anomaly THEN
            SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'Record identified as anomaly and stored in anomalies table';
        END IF;
    END
    """


    cursor.execute("DROP TRIGGER IF EXISTS check_anomaly_before_insert")
    

    cursor.execute(create_trigger)
    print("Trigger created successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    cursor.close()
    db.close()

Anomalies table created successfully!
Trigger created successfully!


In [ ]:
import pandas as pd
import mysql.connector
import time
from datetime import datetime


db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="anomoly"
)

cursor = db.cursor()

try:

    print("Reading test data...")
    df = pd.read_csv('testing_data.csv', delimiter=',', 
                     names=['epoch', 'moteid', 'temperature', 'humidity', 'light', 'voltage'], 
                     skiprows=1,
                     nrows=500)  
    
    total_records = len(df)
    print(f"Total test records to process: {total_records}")

    normal_count = 0
    anomaly_count = 0
    start_time = time.time()
    

    batch_size = 10
    
    for i in range(0, total_records, batch_size):
        batch = df.iloc[i:i+batch_size]
        print(f"\nProcessing batch starting at index {i}")
        
        for cnt, row in batch.iterrows():
            print(f"Processing row {cnt}")
            try:
                cursor.execute("""
                    INSERT INTO sensor_readings 
                    (epoch, moteid, temperature, humidity, light, voltage)
                    VALUES (%s, %s, %s, %s, %s, %s)
                """, (
                    int(row['epoch']),
                    int(row['moteid']),
                    float(row['temperature']),
                    float(row['humidity']),
                    float(row['light']),
                    float(row['voltage'])
                ))
                normal_count += 1
                
            except mysql.connector.Error as err:
                if err.errno == 1644:  
                    anomaly_count += 1
                    print(f"Anomaly detected at row {cnt}")
                else:
                    print(f"Error at row {cnt}: {err}")
        
        
        db.commit()
        current_time = time.time()
        elapsed_time = current_time - start_time
        records_processed = i + len(batch)
        percentage = (records_processed / total_records) * 100
        
        print(f"Progress: {percentage:.1f}% ({records_processed}/{total_records})")
        print(f"Time elapsed: {elapsed_time:.1f} seconds")
        print(f"Normal: {normal_count}, Anomalies: {anomaly_count}")
        print("-" * 50)


    total_time = time.time() - start_time
    print("\nFinal Results:")
    print(f"Total time: {total_time:.1f} seconds")
    print(f"Normal records: {normal_count}")
    print(f"Anomalies: {anomaly_count}")
    print(f"Processing rate: {total_records/total_time:.1f} records/second")

except Exception as e:
    print(f"Error occurred: {e}")
finally:
    cursor.close()
    db.close()

Reading test data...
Total test records to process: 500

Processing batch starting at index 0
Processing row 0
Processing row 1
Processing row 2
Processing row 3
Processing row 4
Processing row 5
Processing row 6
Processing row 7
Processing row 8
Processing row 9
Progress: 2.0% (10/500)
Time elapsed: 14.2 seconds
Normal: 10, Anomalies: 0
--------------------------------------------------

Processing batch starting at index 10
Processing row 10
Processing row 11
Processing row 12
Processing row 13
Processing row 14
Processing row 15
Processing row 16
Processing row 17
Processing row 18
Processing row 19
Progress: 4.0% (20/500)
Time elapsed: 30.0 seconds
Normal: 20, Anomalies: 0
--------------------------------------------------

Processing batch starting at index 20
Processing row 20
Processing row 21
Processing row 22
Processing row 23
Processing row 24
Processing row 25
Processing row 26
Processing row 27
Processing row 28
Processing row 29
Progress: 6.0% (30/500)
Time elapsed: 47.5 s